# A basic template to apply any models to train on the collected data

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn import preprocessing
pd.set_option('display.max_columns', None)

plt.rcParams['figure.figsize'] = [18, 10]




In [12]:
trainDF = pd.read_csv('../outputs/trainXY.csv')

In [13]:
trainDF.head()
trainDF.dropna()

,Unnamed: 0,Unnamed: 0_x,Unnamed: 0_x.1,Unnamed: 0.1,date_implement,policy,metric_change,state_x,policy_type,Unnamed: 0_y,stateName,CEN_stateCode,CEN_Bachelor's Degree or Higher,CEN_Employment Rate,CEN_Hispanic or Latino (of any race),CEN_Median Household Income,CEN_Total Employer Establishments,CEN_Total Households,CEN_Total Housing Units,CEN_Total Population,CEN_Unnamed: 0,CEN_Unnamed: 0.1,CEN_Without Health Care Coverage,CP_0_diff,CP_1_diff,CP_2_diff,FD_adminAndFireResponse,FD_fireResponseOnly,FD_lessThan10kCitizens,FD_moreThan10kCitizens,FD_totalFireDepartments,HB_bed_For-Profit,HB_bed_Non-Profit,HB_bed_State/Local Government,HB_bed_Total,OB_Prevalence,S1_people_fully_vaccinated,S1_people_fully_vaccinated_per_hundred,S1_people_vaccinated,S1_people_vaccinated_per_hundred,S2_people_fully_vaccinated,S2_people_fully_vaccinated_per_hundred,S2_people_vaccinated,S2_people_vaccinated_per_hundred,S3_people_fully_vaccinated,S3_people_fully_vaccinated_per_hundred,S3_people_vaccinated,S3_people_vaccinated_per_hundred,S4_people_fully_vaccinated,S4_people_fully_vaccinated_per_hundred,S4_people_vaccinated,S4_people_vaccinated_per_hundred,SP_Governor Political Affiliation,SP_State Attorney General Political Affiliation,SP_State House Majority Political Affiliation,SP_State Senate Majority Political Affiliation,Unnamed: 0_y.1,submission_date,state_y,new_case,caseInterpolate_MA7,caseInterpolate_savitzky31_3,caseInterpolate_gauss8,caseInterpolate_gauss3,new_case_zscore,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,0,0,0,0,2020-03-13,C1_School closing,1.0,Alaska,0,1.0,Alaska,AK,30.2,59.3,49824.0,75463.0,21399.0,252199.0,326200.0,733391.0,1.0,0.0,12.2,230.0,353.0,655.0,181.0,0.0,181.0,8.0,189.0,0.36,1.55,0.29,2.20,31.9,57117.0,7.81,126381.0,17.28,268405.0,36.69,312535.0,42.72,335664.0,45.88,381619.0,52.17,391118.0,53.46,446329.0,61.01,0.0,0.0,0.0,0.0,32839.0,2020-03-13,AK,1.0,0.142857,-21.030578,2.0,0.0,-0.752799,1.0,0.010204,0.001319,1.0,0.006803,0.001319,1.0,0.005102,0.001319,1.0,0.142857,0.041538,1.0,0.095238,0.041538,1.0,0.071429,0.041538,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,1,1,1,1,2020-03-16,C1_School closing,2.0,Alaska,0,1.0,Alaska,AK,30.2,59.3,49824.0,75463.0,21399.0,252199.0,326200.0,733391.0,1.0,0.0,12.2,230.0,353.0,655.0,181.0,0.0,181.0,8.0,189.0,0.36,1.55,0.29,2.20,31.9,57117.0,7.81,126381.0,17.28,268405.0,36.69,312535.0,42.72,335664.0,45.88,381619.0,52.17,391118.0,53.46,446329.0,61.01,0.0,0.0,0.0,0.0,5015.0,2020-03-16,AK,0.0,0.142857,21.332012,3.0,1.0,-0.756350,1.0,0.010204,0.004615,1.0,0.006803,0.004615,1.0,0.005102,0.004615,1.0,0.214286,0.090000,1.0,0.142857,0.090000,1.0,0.107143,0.090000,1.0,0.071429,0.017692,1.0,0.047619,0.017692,1.0,0.035714,0.017692
2,2,2,2,2,2020-03-12,C2_Workplace closing,1.0,Alaska,0,1.0,Alaska,AK,30.2,59.3,49824.0,75463.0,21399.0,252199.0,326200.0,733391.0,1.0,0.0,12.2,230.0,353.0,655.0,181.0,0.0,181.0,8.0,189.0,0.36,1.55,0.29,2.20,31.9,57117.0,7.81,126381.0,17.28,268405.0,36.69,312535.0,42.72,335664.0,45.88,381619.0,52.17,391118.0,53.46,446329.0,61.01,0.0,0.0,0.0,0.0,3062.0,2020-03-12,AK,0.0,0.0000

In [14]:
# Finalizing Indexes so we can drop state names 
trainDF = trainDF.reset_index(drop=True)

# Get a list of columns with features 
columns = trainDF.columns.to_list()

In [15]:
Y_columns = [x for x in columns if 'Y_' in x]

In [16]:
X_columns = [x for x in columns if 'Y_' not in x and 'Unnamed' not in x]

X_columns.remove('date_implement')
# X_columns.remove('policy')
X_columns.remove('state_x')
# X_columns.remove('policy_type')
X_columns.remove('stateName')
X_columns.remove('CEN_stateCode')
X_columns.remove('submission_date')
X_columns.remove('state_y')

X_columns.remove('caseInterpolate_gauss3')
X_columns.remove('caseInterpolate_MA7')
X_columns.remove('caseInterpolate_savitzky31_3')
X_columns.remove('caseInterpolate_gauss8')
X_columns.remove('new_case_zscore')
X_columns.remove('new_case')



In [17]:
X = trainDF[X_columns]
y = trainDF[Y_columns]


In [18]:
y

,Y_TREND_caseInterpolate_MA7_14,Y_TRENDQUANT_caseInterpolate_MA7_14,Y_SLOPE_caseInterpolate_MA7_14,Y_TREND_caseInterpolate_MA7_21,Y_TRENDQUANT_caseInterpolate_MA7_21,Y_SLOPE_caseInterpolate_MA7_21,Y_TREND_caseInterpolate_MA7_28,Y_TRENDQUANT_caseInterpolate_MA7_28,Y_SLOPE_caseInterpolate_MA7_28,Y_TREND_caseInterpolate_gauss8_14,Y_TRENDQUANT_caseInterpolate_gauss8_14,Y_SLOPE_caseInterpolate_gauss8_14,Y_TREND_caseInterpolate_gauss8_21,Y_TRENDQUANT_caseInterpolate_gauss8_21,Y_SLOPE_caseInterpolate_gauss8_21,Y_TREND_caseInterpolate_gauss8_28,Y_TRENDQUANT_caseInterpolate_gauss8_28,Y_SLOPE_caseInterpolate_gauss8_28,Y_TREND_caseInterpolate_gauss3_14,Y_TRENDQUANT_caseInterpolate_gauss3_14,Y_SLOPE_caseInterpolate_gauss3_14,Y_TREND_caseInterpolate_gauss3_21,Y_TRENDQUANT_caseInterpolate_gauss3_21,Y_SLOPE_caseInterpolate_gauss3_21,Y_TREND_caseInterpolate_gauss3_28,Y_TRENDQUANT_caseInterpolate_gauss3_28,Y_SLOPE_caseInterpolate_gauss3_28
0,1.0,0.010204,0.001319,1.0,0.006803,0.001319,1.0,0.005102,0.001319,1.0,0.142857,0.041538,1.0,0.095238,0.041538,1.0,0.071429,0.041538,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,1.0,0.010204,0.004615,1.0,0.006803,0.004615,1.0,0.005102,0.004615,1.0,0.214286,0.090000,1.0,0.142857,0.090000,1.0,0.107143,0.090000,1.0,0.071429,0.017692,1.0,0.047619,0.017692,1.0,0.035714,0.017692
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,1.0,0.071429,0.025385,1.0,0.047619,0.025385,1.0,0.035714,0.025385,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,-1.0,-0.571429,-0.324176,-1.0,-0.380952,-0.324176,-1.0,-0.153061,-0.324176,-1.0,-0.357143,-0.277692,-1.0,-0.285714,-0.277692,-1.0,-0.142857,-0.277692,-1.0,-0.571429,-0.390769,-1.0,-0.380952,-0.390769,-1.0,-0.285714,-0.390769
4,1.0,0.010204,0.001319,1.0,0.006803,0.001319,1.0,0.005102,0.001319,1.0,0.142857,0.041538,1.0,0.095238,0.041538,1.0,0.071429,0.041538,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,1.0,1.020408,0.390659,1.0,0.918367,0.390659,1.0,0.775510,0.390659,1.0,0.428571,0.406923,1.0,0.380952,0.406923,1.0,0.392857,0.406923,1.0,0.928571,0.603077,1.0,0.666667,0.603077,1.0,0.607143,0.603077
1784,-1.0,-0.173469,0.269890,1.0,0.013605,0.269890,-1.0,-0.066327,0.269890,0.0,0.000000,-0.065385,-1.0,-0.047619,-0.065385,-1.0,-0.250000,-0.065385,-1.0,-0.357143,0.093077,-1.0,-0.095238,0.093077,-1.0,-0.142857,0.093077
1785,-1.0,-5.612245,-3.648681,-1.0,-4.489796,-3.648681,-1.0,-4.030612,-3.648681,-1.0,-5.285714,-4.459231,-1.0,-4.714286,-4.459231,-1.0,-4.250000,-4.459231,-1.0,-6.357143,-4.703846,-1.0,-5.000000,-4.703846,-1.0,-4.035714,-4.703846
1786,-1.0,-6.826531,-9.145055,-1.0,-6.414966,-9.145055,-1.0,-10.326531,-9.145055,-1.0,-1.142857,-5.183846,-1.0,-4.095238,-5.183846,-1.0,-6.392857,-5.183846,1.0,2.071429,-5.693846,-1.0,-3.142857,-5.693846,-1.0,-5.785714,-5.693846


# Encoding Policy Names 

In [19]:
X['policy'].unique()

array(['C1_School closing', 'C2_Workplace closing',
       'C3_Cancel public events', 'C4_Restrictions on gatherings',
       'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls',
       'H1_Public information campaigns', 'H2_Testing policy',
       'H3_Contact tracing', 'H6_Facial Coverings',
       'H7_Vaccination policy'], dtype=object)

In [20]:
le = preprocessing.LabelEncoder()
le.fit(X['policy'])
print(le.classes_)

['C1_School closing' 'C2_Workplace closing' 'C3_Cancel public events'
 'C4_Restrictions on gatherings' 'C5_Close public transport'
 'C6_Stay at home requirements' 'C7_Restrictions on internal movement'
 'C8_International travel controls' 'H1_Public information campaigns'
 'H2_Testing policy' 'H3_Contact tracing' 'H6_Facial Coverings'
 'H7_Vaccination policy']


In [21]:
X['policy'] = pd.Series(le.transform(X['policy']))
original_policies_decoded = pd.Series(le.inverse_transform(X['policy']))

/var/folders/66/w1588p9n1654b1yky4q5nz580000gn/T/ipykernel_5700/2867200982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['policy'] = pd.Series(le.transform(X['policy']))


------

-------

# Train Models Here